# Create a MatrixTable and QC the hell out of it
## Import stuff and set your parameters
First, we import necessary libraries and configurations from config.toml. Then we initialise Spark and Hail. 

In [ ]:
!pip install toml
!pip install jupytext

In [2]:
# Imports
import toml
from pathlib import Path
from datetime import datetime
from pprint import pprint
import sys
from distutils.version import LooseVersion
import subprocess
import os

import pandas as pd
import pyspark
import dxpy
import dxdata
import hail as hl
    
module_path = Path('..').resolve().__str__()

if module_path not in sys.path:
    sys.path.append(module_path)

try:
    os.mkdir("../tmp")
except:
    pass
    
from src.utils import get_position, lookup_vcfs
from src.matrixtables import *
from src.resources import lauryns_variants


hl.plot.output_notebook()

Loading BokehJS ...

In [162]:
# Parameters
with open("../config.toml") as f:
    conf = toml.load(f)

IMPORT = conf["IMPORT"]
GENE = conf["GENE"]
VCF_VERSION = IMPORT["VCF_VERSION"]
REFERENCE_GENOME = conf["REFERENCE_GENOME"]
DATABASE = IMPORT["DATABASE"]


LOG_FILE = Path(IMPORT["LOG_DIR"],f"{GENE}_{datetime.now().strftime('%H%M')}.log").resolve().__str__()
MAP_FILE = Path(IMPORT["MAPPING_FILE"]).resolve().__str__()
INT_FILE = Path(IMPORT["INTERVAL_FILE"]).resolve().__str__()

VCF_DIR = Path(IMPORT["VCF_DIR"]).resolve().__str__()

DOWNSAMPLE_P = IMPORT.get("DOWNSAMPLE_P", None)

SNV_ONLY = conf["ANNOTATE"]["SNV_ONLY"]
USE_VEP = conf["ANNOTATE"]["USE_VEP"]
MISSENSE_ONLY = conf["ANNOTATE"]["MISSENSE_ONLY"]

VEP_JSON = Path(conf["ANNOTATE"]["VEP_JSON"]).resolve().__str__()

ANNOTATION_DIR = conf["ANNOTATE"]["ANNOTATION_DIR"]
ANNOTATION_FILE = Path(ANNOTATION_DIR, f"{GENE}.tsv").resolve().__str__()

MIN_DP = conf["ENTRY_QC"]["MIN_DP"]
MIN_GQ = conf["ENTRY_QC"]["MIN_GQ"]
MIN_PL = conf["ENTRY_QC"]["MIN_PL"]

MIN_P_HWE = conf["VARIANT_QC"]["MIN_P_HWE"]
MIN_VAR_GQ = conf["VARIANT_QC"]["MIN_VAR_GQ"]

MIN_CALL_RATE = conf["SAMPLE_QC"]["MIN_CALL_RATE"]
MIN_MEAN_DP = conf["SAMPLE_QC"]["MIN_MEAN_DP"]
MIN_MEAN_GQ = conf["SAMPLE_QC"]["MIN_MEAN_GQ"]

REL_PATH = Path(conf["SAMPLE_QC"]["DATA_DIR"], conf["SAMPLE_QC"]["REL_FILE"]).resolve().__str__()
SAMP_PATH = Path(conf["SAMPLE_QC"]["DATA_DIR"], conf["SAMPLE_QC"]["SAMP_FILE"]).resolve().__str__()

TMP_DIR = conf["EXPORT"]["TMP_DIR"]

BGEN_FILE = Path(TMP_DIR, f"{GENE}").resolve().__str__()
ANNOTATIONS_FILE = Path(TMP_DIR, f"{GENE}.annotations").resolve().__str__()
SETLIST_FILE = Path(TMP_DIR, f"{GENE}.setlist").resolve().__str__()


In [4]:
# Spark and Hail

sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

try:
    mt_database = dxpy.find_one_data_object(name=DATABASE)["id"]
except DXSearchError:
    spark.sql(f"CREATE DATABASE {DATABASE} LOCATION  'dnax://'")
    mt_database = dxpy.find_one_data_object(name=DATABASE)["id"]

hl.init(sc=sc, default_reference=REFERENCE_GENOME, log=LOG_FILE, tmp_dir=f'dnax://{mt_database}/tmp/')

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.4
SparkUI available at http://ip-10-60-72-206.eu-west-2.compute.internal:8081
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /opt/notebooks/gogoGPCR/hail_logs/DRD2_0822.log


In [5]:
# VCFs
mapping = pd.read_csv(MAP_FILE, sep = "\t").set_index("HGNC", drop=False).loc[GENE,:].to_dict()
vcf_files = lookup_vcfs(mapping = mapping, vcfdir = VCF_DIR, gene = GENE, version = VCF_VERSION)

In [7]:
# Import
mt = import_mt(vcf_files.get("vcfs"), mapping).key_rows_by("locus", "alleles")#.checkpoint(checkpoint_file)

v, s = mt.count()
pprint(f"{v} variants and {s} samples after import")

'582 variants and 200643 samples after import'


In [8]:
# Checkpoint
stage = "raw"
checkpoint_file = f"/tmp/{GENE}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite = True)

2021-10-08 08:27:58 Hail: INFO: Coerced sorted dataset
2021-10-08 08:29:56 Hail: INFO: wrote matrix table with 582 rows and 200643 columns in 1 partition to /tmp/DRD2.raw.cp.mt
    Total size: 602.72 MiB
    * Rows/entries: 601.41 MiB
    * Columns: 1.31 MiB
    * Globals: 11.00 B
    * Smallest partition: 582 rows (601.41 MiB)
    * Largest partition:  582 rows (601.41 MiB)


In [243]:
#mt = hl.read_matrix_table(checkpoint_file)

In [9]:
# Downsample
if DOWNSAMPLE_P is not None:
    mt = downsample_mt(mt, DOWNSAMPLE_P)

    pprint(f"{mt.count_cols()} samples after downsampling")

In [10]:
# Interval QC
mt = interval_qc_mt(mt, mapping, "file:" + INT_FILE)

pprint(f"{mt.count_rows()} variants after interval filtering")

2021-10-08 08:31:25 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)
2021-10-08 08:31:28 Hail: INFO: Coerced sorted dataset


'276 variants after interval filtering'


In [11]:
# Split multi
mt = mt.filter_rows(mt.alleles.length() <= 6)
mt = smart_split_multi_mt(mt)

pprint(f'{mt.count_rows()} variants with not more than 6 alleles after splitting')

2021-10-08 08:31:38 Hail: INFO: Coerced sorted dataset
2021-10-08 08:31:42 Hail: INFO: Coerced sorted dataset
2021-10-08 08:31:45 Hail: INFO: Coerced sorted dataset


'297 variants with not more than 6 alleles after splitting'


In [13]:
if SNV_ONLY:
    mt = mt.filter_rows(hl.len(mt.alleles[1]) == 1)
    
    pprint(f"{mt.count_rows()} SNVs")

2021-10-08 08:32:34 Hail: INFO: Coerced sorted dataset
2021-10-08 08:32:35 Hail: INFO: Coerced sorted dataset
2021-10-08 08:32:36 Hail: INFO: Coerced sorted dataset


'297 SNVs'


In [15]:
if USE_VEP:
    mt = hl.vep(mt, "file:" + VEP_JSON)
    
    arr = mt.vep.transcript_consequences.mane_select.take(1)[0] # take one MANE SELECT array
    
    for i, transcript in enumerate(arr): # hope that all MS transcripts have same index
        if transcript is not None:
            break
    
    found_MANE_SELECT = mt.aggregate_rows(hl.agg.all(hl.is_defined(mt.vep.transcript_consequences.mane_select[i]))) # sanity check
    
    assert found_MANE_SELECT, "Careful! Could not locate MANE SELECT transcript. Probably good idea to check manually"
    

2021-10-08 08:33:38 Hail: INFO: Coerced sorted dataset
2021-10-08 08:33:39 Hail: INFO: Coerced sorted dataset
2021-10-08 08:33:41 Hail: INFO: Coerced sorted dataset
2021-10-08 08:33:42 Hail: INFO: Coerced sorted dataset
2021-10-08 08:33:43 Hail: INFO: Coerced sorted dataset
2021-10-08 08:33:45 Hail: INFO: Coerced sorted dataset


In [77]:
if MISSENSE_ONLY:
    
    assert USE_VEP, "Filtering missense variants requires USE_VEP flag"
    
    mt = mt.filter_rows(mt.vep.most_severe_consequence == "missense_variant")
    mt = mt.filter_rows(mt.vep.transcript_consequences.consequence_terms[i][0] == "missense_variant")
    mt = mt.annotate_rows(protCons = mt.vep.transcript_consequences.amino_acids[i].split("/")[0]  + hl.str(mt.vep.transcript_consequences.protein_end[i]) + mt.vep.transcript_consequences.amino_acids[i].split("/")[1])
    
    pprint(f"{mt.count_rows()} missense variant")

2021-10-08 09:15:24 Hail: INFO: Coerced sorted dataset
2021-10-08 09:15:25 Hail: INFO: Coerced sorted dataset
2021-10-08 09:15:27 Hail: INFO: Coerced sorted dataset


'149 missense variant'


In [97]:
import pandas as pd
#df = pd.read_csv("/opt/notebooks/gogoGPCR/tmp/211004_drd2_variants_aggregation.tsv", sep = "\t")
ht = hl.import_table("/211004_drd2_variants_aggregation.tsv", impute = True).key_by("Variant")
ht.show()

2021-10-08 09:40:28 Hail: INFO: Reading table to impute column types
2021-10-08 09:40:28 Hail: INFO: Loading 50 fields. Counts by type:
  str: 41
  int32: 9


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Var,Gi1_activation rate_mean_100qm,Gi1_activation rate_mean_10qm,Gi1_activation rate_pval_100qm,Gi1_activation rate_pval_10qm,Gi1_amplitude_mean_100qm,Gi1_amplitude_mean_10qm,Gi1_amplitude_pval_100qm,Gi1_amplitude_pval_10qm,GoA_activation rate_mean_100qm,GoA_activation rate_mean_10qm,GoA_activation rate_pval_100qm,GoA_activation rate_pval_10qm,GoA_amplitude_mean_100qm,GoA_amplitude_mean_10qm,GoA_amplitude_pval_100qm,GoA_amplitude_pval_10qm,Gz_activation rate_mean_100qm,Gz_activation rate_mean_10qm,Gz_activation rate_pval_100qm,Gz_activation rate_pval_10qm,Gz_amplitude_mean_100qm,Gz_amplitude_mean_10qm,Gz_amplitude_pval_100qm,Gz_amplitude_pval_10qm,Labcode,Variant,SurfaceExpression,SurfaceExpression_sem,SurfaceExpression_n,SurfaceExpression_p,TotalExpression,TotalExpression_sem,TotalExpression_n,TotalExpression_p,AA consequence,Transcript Consequence,sequence number for sorting,SequenceNumber,GPCRdb,Segment,WTaa,NMaa,GPCRdb_short,number_of_impairments,number_of_impairments_Gi1,number_of_impairments_GoA,number_of_impairments_Gz,number_of_across_pathways,mean_amplitudes
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,int32,str,str,str,int32,str,str,str,int32,int32,str,str,str,str,str,int32,int32,int32,int32,int32,str
"""A127V 3x45""","""0,597015370283226""","""0,643574119127666""","""0,1727""","""0,9911""","""69,5316220360144""","""63,1300660485471""","""0,0924""","""0,4849""","""1,58441401105866""","""1,80985162070555""","""0,5239""","""0,9911""","""96,103788315811""","""101,594723211329""","""0,9995""","""0,9999""","""0,0233778347710404""","""0,020530791018249""","""0,9521""","""852""","""90,7583896418041""","""88,0322811714467""","""0,9907""","""999""","""DRD2MT123""","""A127V""","""43,3395618160454""","""10,0578815197535""",5,"""0,1592""","""51,7147173783332""","""12,2691063843954""",5,"""0,3719""","""A127V""","""c.380C>T""",127,127,"""3.45x45""","""TM3""","""A""","""V""","""3x45""",0,0,0,0,0,"""84,8584784041587"""
"""A177T ""","""0,910903915387408""","""0,769881694191999""","""0,9985""","""0,9989""","""112,222738456505""","""98,3280001795086""","""0,9998""","""0,9996""","""1,908217378732""","""1,97519010909049""","""0,9645""","""0,9989""","""74,1389680451515""","""65,8438635251985""","""0,9917""","""0,9731""","""0,043365611478242""","""0,0363208119655175""","""0,9999""","""0,9997""","""89,6257054469003""","""89,7260401721463""","""0,9823""","""0,9991""","""DRD2MT26""","""A177T""","""99,7637678729382""","""8,3466360712182""",5,"""0,9995""","""90,2048452624591""","""8,40297787331943""",5,"""0,9999""","""A177T""","""c.529G>A""",177,177,"""""","""ECL2""","""A""","""T""","""""",0,0,0,0,0,"""88,314219304235"""
"""A273V ""","""1,18461745863355""","""1,03470264977696""","""0,9993""","""0,9996""","""83,0794661236873""","""79,6397858275958""","""0,7158""","""991""","""2,50006045958493""","""2,38950668556777""","""0,9994""","""0,9996""","""79,2166969786639""","""94,3691372591675""","""0,9983""","""0,9994""","""0,0485121486190895""","""0,0409948056232525""","""0,9994""","""0,9997""","""101,108286184116""","""91,6896599288242""","""0,9988""","""0,9993""","""DRD2MT59""","""A273V""","""71,4000533739933""","""5,02774873947585""",4,"""0,9982""","""64,6226063570699""","""12,5391044137044""",4,"""0,9784""","""A273V""","""c.818C>T""",273,273,"""""","""ICL3""","""A""","""V""","""""",0,0,0,0,0,"""88,1838387170091"""
"""A29V ""","""1,19648135362991""","""0,687120249512224""","""0,9992""","""0,9982""","""71,9837467989575""","""67,1176980248258""","""0,1488""","""0,6821""","""1,94605727233454""","""2,11776576694169""","""0,9917""","""0,9982""","""61,9164152821397""","""67,6666142497236""","""0,9556""","""978""","""0,0385379042494758""","""0,0356496203232574""","""0,9995""","""0,9997""","""100,074058345824""","""103,838894340631""","""0,9986""","""0,9999""","""DRD2MT11""","""A29V""","""75,2243139899891""","""8,67074687167099""",5,"""0

In [111]:
mt = mt.annotate_rows(annotations = ht[mt.protCons])

In [131]:
def generate_report(mt):
    intr = mt.filter_rows((hl.is_defined(mt.annotations)) & (hl.agg.any(mt.GT.is_non_ref())))
    intr = hl.variant_qc(intr)
    intr = intr.select_rows(intr.variant_qc, intr.protCons, intr.annotations).rows()
    intr = intr.annotate(**intr.variant_qc)
    intr = intr.drop("variant_qc", "gq_stats", "dp_stats")
    
    return intr

intr = generate_report(mt)
intr.export("/opt/notebooks/tmp/DRD2_before_QC.tsv")

2021-10-08 10:10:00 Hail: INFO: Coerced sorted dataset
2021-10-08 10:10:01 Hail: INFO: Coerced sorted dataset
2021-10-08 10:10:15 Hail: INFO: Coerced sorted dataset
2021-10-08 10:10:16 Hail: INFO: Coerced sorted dataset
2021-10-08 10:10:17 Hail: INFO: Coerced sorted dataset
2021-10-08 10:10:18 Hail: INFO: Coerced sorted dataset
2021-10-08 10:10:20 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-10-08 10:10:20 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-10-08 10:10:20 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-10-08 10:10:48 Hail: INFO: merging 2 files totalling 61.2K...
2021-10-08 10:10:48 Hail: INFO: while writing:
    /opt/notebooks/tmp/DRD2_before_QC.tsv
  merge time: 50.468ms


In [307]:
mt = annotate_mt(mt = mt, gene = GENE, annotations = "file:" + ANNOTATION_FILE)

interesting = mt.filter_rows((hl.is_defined(mt.annotations)) & (hl.agg.any(mt.GT.is_non_ref()))).count_rows()
pprint(f"{interesting} annotated variants found before QC")

AttributeError: module 'src.annotations' has no attribute 'annotate_GCGR'

In [134]:
# Genotype GQ
mt = genotype_filter_mt(mt, MIN_DP, MIN_GQ, True)

missing = mt.aggregate_entries(hl.agg.sum(~hl.is_defined(mt.GT)))
pprint(f"{missing} missing or filtered entries after Call QC")

2021-10-08 10:05:30 Hail: INFO: Coerced sorted dataset
2021-10-08 10:05:31 Hail: INFO: Coerced sorted dataset
2021-10-08 10:05:48 Hail: INFO: Coerced sorted dataset


'37 missing or filtered entries after Call QC'


In [135]:
# Checkpoint
stage = "QC1"
checkpoint_file = f"/tmp/{GENE}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite = True)

2021-10-08 10:06:33 Hail: INFO: Coerced sorted dataset
2021-10-08 10:06:33 Hail: INFO: Coerced sorted dataset
2021-10-08 10:06:51 Hail: INFO: Coerced sorted dataset
2021-10-08 10:07:33 Hail: INFO: Coerced sorted dataset
2021-10-08 10:07:34 Hail: INFO: Coerced sorted dataset
2021-10-08 10:07:36 Hail: INFO: Coerced sorted dataset
2021-10-08 10:07:38 Hail: INFO: Coerced sorted dataset
2021-10-08 10:07:39 Hail: INFO: Coerced sorted dataset
2021-10-08 10:07:57 Hail: INFO: Coerced sorted dataset
2021-10-08 10:07:58 Hail: INFO: Coerced sorted dataset
2021-10-08 10:07:58 Hail: INFO: Coerced sorted dataset
2021-10-08 10:08:00 Hail: INFO: Coerced sorted dataset
2021-10-08 10:08:01 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-10-08 10:08:01 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-10-08 10:08:02 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-10-08 10:08:02 Hail: INFO: Coerced sorted dataset
2021-10-08 10:08:03 Hail: INFO: Coerced sorted d

In [167]:
# Re-load
stage = "QC1"
checkpoint_file = f"/tmp/{GENE}.{stage}.cp.mt"

mt = hl.read_matrix_table(checkpoint_file)

In [168]:
# Variant QC
mt = variant_QC_mt(mt, MIN_P_HWE, MIN_VAR_GQ)

interesting = mt.filter_rows((hl.is_defined(mt.annotations)) & (hl.agg.any(mt.GT.is_non_ref()))).count_rows()
print(f"{mt.count_rows()} variants remaining after QC of which {interesting} are annotated")

149 variants remaining after QC of which 101 are annotated


In [169]:
# Withdrawn
mt = mt.filter_cols(~mt.s.startswith("W"))

print(f"Samples remaining after removing withdrawn participants: {mt.count_cols()} ")

Samples remaining after removing withdrawn participants: 200611 


In [170]:
# Sample QC
mt = sample_QC_mt(mt, MIN_CALL_RATE, MIN_MEAN_DP, MIN_MEAN_GQ)

print(f"Samples remaining after QC: {mt.count_cols()} ")

Samples remaining after QC: 187645 


In [171]:
# Related
rel = hl.import_table("file:" + REL_PATH, key = "eid")
mt = mt.anti_join_cols(rel)

print(f"Samples remaining after removing related samples: {mt.count_cols()} ")

2021-10-08 10:41:59 Hail: INFO: Reading table without type imputation
  Loading field 'eid' as type str (not specified)


Samples remaining after removing related samples: 174422 


In [172]:
# Hard filters
samp = hl.import_table("file:" + SAMP_PATH, key = "eid")
mt = mt.semi_join_cols(samp)

print(f"Samples remaining after after hard filters: {mt.count_cols()} ")

2021-10-08 10:42:09 Hail: INFO: Reading table without type imputation
  Loading field 'eid' as type str (not specified)


Samples remaining after after hard filters: 144817 


In [173]:
mt = mt.filter_rows((mt.variant_qc.AF[0] > 0.0) & (mt.variant_qc.AF[0] < 1.0))

interesting = mt.filter_rows((hl.is_defined(mt.annotations)) & (hl.agg.any(mt.GT.is_non_ref()))).count_rows()
print(f"{mt.count_rows()} variants remaining after QC of which {interesting} are annotated")

149 variants remaining after QC of which 75 are annotated


In [174]:
STAGE = "final"
WRITE_PATH = "dnax://" + mt_database + f"/{GENE}.{STAGE}.mt"

mt.write(WRITE_PATH)

2021-10-08 10:44:25 Hail: INFO: wrote matrix table with 149 rows and 144817 columns in 2 partitions to dnax://database-G5B4q0jJ860jjqP2Jq6pfvF5/DRD2.final.mt
    Total size: 68.33 MiB
    * Rows/entries: 63.15 MiB
    * Columns: 5.18 MiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  149 rows (63.15 MiB)


In [177]:
def generate_report(mt):
    intr = mt.filter_rows((hl.is_defined(mt.annotations)) & (hl.agg.any(mt.GT.is_non_ref())))
    intr = hl.variant_qc(intr)
    intr = intr.select_rows(intr.variant_qc, intr.protCons, intr.annotations).rows()
    intr = intr.annotate(**intr.variant_qc, **intr.annotations)
    intr = intr.drop("variant_qc", "gq_stats", "dp_stats", "annotations")
    
    return intr

intr = generate_report(mt)
intr.export("/opt/notebooks/tmp/DRD2_after_QC.tsv")

2021-10-08 10:47:42 Hail: INFO: merging 2 files totalling 45.8K...
2021-10-08 10:47:42 Hail: INFO: while writing:
    /opt/notebooks/tmp/DRD2_after_QC.tsv
  merge time: 60.437ms


In [164]:
STAGE = "final"
WRITE_PATH = "dnax://" + mt_database + f"/{GENE}.{STAGE}.mt"

mt = hl.read_matrix_table(WRITE_PATH)

In [340]:
# BGEN
write_bgen(mt, "file:" + BGEN_FILE)

2021-09-21 16:21:57 Hail: INFO: while writing:
    file:/opt/notebooks/gogoGPCR/tmp/GCGR.bgen
  merge time: 137.636ms


In [357]:
# ANNOTATIONS

mt = add_varid(mt)

annotations = (
    mt.select_rows(
        varid = mt.varid,
        gene = mt.vep.transcript_consequences.gene_symbol[0],
        annotation = mt.annotation
    )
    .rows()
    .key_by("varid")
    .drop("locus")
    .drop("alleles")
)
annotations.export("file:" + ANNOTATIONS_FILE, header=False)

2021-09-21 16:32:15 Hail: INFO: Coerced sorted dataset
2021-09-21 16:32:24 Hail: INFO: merging 2 files totalling 4.7K...
2021-09-21 16:32:24 Hail: INFO: while writing:
    file:/opt/notebooks/gogoGPCR/tmp/GCGR.annotations
  merge time: 16.836ms


In [360]:
# SETLIST
position = mt.aggregate_rows(hl.agg.min(mt.locus.position))
names = mt.varid.collect()
names_str = ",".join(names)

line = f"{mt.vep.transcript_consequences.gene_symbol[0].collect()[0]}\t{mt.locus.contig.collect()[0]}\t{position}\t{names_str}"

with open(SETLIST_FILE, "w") as f:
    f.write(line)

In [361]:
bgen_file = BGEN_FILE + ".bgen"
sample_file = BGEN_FILE + ".sample"

subprocess.run(["dx", "upload", bgen_file, sample_file, ANNOTATIONS_FILE, SETLIST_FILE, "--path", "/data/burden/"], check = True, shell = False)

CompletedProcess(args=['dx', 'upload', '/opt/notebooks/gogoGPCR/tmp/GCGR.bgen', '/opt/notebooks/gogoGPCR/tmp/GCGR.sample', '/opt/notebooks/gogoGPCR/tmp/GCGR.annotations', '/opt/notebooks/gogoGPCR/tmp/GCGR.setlist', '--path', '/data/burden/'], returncode=0)